In [1]:
import pymongo
from pymongo import MongoClient
import tqdm
from collections import defaultdict, Counter
import json

client = MongoClient('localhost', 27017)
db = client.istex
cambridge = db.cambridge
springer = db.springer
total = springer.find({'categories': {'$exists': True}, 'abstract': {'$exists': True}, 'title': {'$exists': True}}).count() + cambridge.find({'categories': {'$exists': True}, 'abstract': {'$exists': True}, 'title': {'$exists': True}}).count()
categories = defaultdict(set)

cnt = Counter()

with open('springer_cambridge.txt', 'a') as outfile:
    for col in [springer, cambridge]:
        for i in tqdm.tqdm(col.find({'categories': {'$exists': True}, 'abstract': {'$exists': True}, 'title': {'$exists': True}, 'language': 'eng'})):
            for ar, v in i['categories'].iteritems():
                if ar == 'scopus':
                    cat = set()
                    for el in v:
                        if '1 -' in el:
                            cat.add(el)
                    cat = list(cat)
                    json.dump({'abstract': i['abstract'], 'label': ','.join(cat), 'id': i['id'], 'title': i['title']}, outfile)
                    outfile.write('\n')


51805it [00:34, 1520.41it/s]
96642it [00:53, 1816.35it/s]


In [27]:
import pandas as pd

d = pd.read_json("springer_cambridge.txt", orient='records', lines=True)
d = d[d.label != '']

In [28]:
d.count()

abstract    261462
id          261462
label       261462
title       261462
dtype: int64

In [29]:
d.labels = d.label.apply(lambda x: x.split(','))
d.labels[0]

[u'1 - Physical Sciences', u'1 - Life Sciences']

In [42]:
import numpy as np
d['health_sciences'] = np.full((d.shape[0],), 0)
d['life_sciences'] = np.full((d.shape[0],), 0)
d['physical_sciences'] = np.full((d.shape[0],), 0)
d['social_sciences'] = np.full((d.shape[0],), 0)

health_mask = d.label.str.contains('1 - Health Sciences')
physical_mask = d.label.str.contains('1 - Physical Sciences')
life_mask = d.label.str.contains('1 - Life Sciences')
social_mask = d.label.str.contains('1 - Social Sciences')

d['health_sciences'][health_mask] = 1
d['life_sciences'][life_mask] = 1
d['social_sciences'][social_mask] = 1
d['physical_sciences'][physical_mask] = 1
d

/home/sami/.virtualenvs/jupyter/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/home/sami/.virtualenvs/jupyter/lib/python2.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/sami/.virtualenvs/jupyter/lib/python2.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sami/.virtualenvs/jupyter/lib/python2.7/site-pack

,abstract,id,label,title,health_sciences,life_sciences,physical_sciences,social_sciences
0,Abstract: Uptake and metabolic turnover of a v...,91543AB49EF6104156B755A81D4D3E11B1E3CF19,"1 - Physical Sciences,1 - Life Sciences",Activity of marine psychrophilic bacteria at e...,0,1,1,0
1,"Summary: The calcium antagonist falipamil, a c...",EFB951031543D6D1B8479250A5B6BBAB6A7B24C8,"1 - Life Sciences,1 - Health Sciences",Atropine and verapamil interactions in healthy...,1,1,0,0
2,Summary: Experiments were performed to compare...,9DEA4ED385B3B941293CBADB71A3004AA4BFA390,1 - Life Sciences,Mechanism of asymmetric block of K channels by...,0,1,0,0
3,Abstract: Cells of the coccolithophoridCricosp...,152CF0B3C7000B08F84612084EE4CC844F692482,"1 - Physical Sciences,1 - Life Sciences",Ultrastructural study and calcium and cadmium ...,0,1,1,0
4,Abstract: A calcium requirement was shown for ...,0866A00EDFAA77712E45BCCAFEFC042E296D3772,"1 - Life Sciences,1 - Health Sciences",The calcium requirement for functional vesicle...,1,1,0,0
5,Abstract: To any multiplicative cohomology the...,120565D2411839054DA3C6D823478406BF00460D,1 - Physical Sciences,Realisierung formaler Gruppen durch Kohomologi...,0,0,1,0
6,Abstract: LetT be a one-to-one mapping ofn-dim...,71734F49520DB1B0C43FE01EF888C47B2DBCCB6B,1 - Physical Sciences,The structure of space-time transformations,0,0,1,0
7,Summary: Circumcision is a religious tradition...,F9FC53EC20399CA5874E1CF4B7A81E90733E2CDD,1 - Health Sciences,The management of post-circumcision penile con...,1,0,0,0
8,Zusammenfassung: Es wurden 30%ige drittgradige...,9B011803A6EB7CD83AF214DCCB6B65512FC2477E,1 - Health Sciences,Experimente zur lokalen Verbrennungsbehandlung,1,0,0,0
9,Abstract: In the proposed system for genetic n...,0D4698BEB788E2D7407A1CE453062D2648905CD6,1 - Life Sciences,Genetic nomenclature for the immunoglobulin lo...,0,1,0,0


In [43]:
d = d.drop("label", axis=1)

In [44]:
d.to_csv('scopus.csv', encoding='utf-8', index=False)

In [64]:
from sklearn.model_selection import train_test_split


train, test = train_test_split(d, test_size=0.2)

In [65]:
train.to_csv('scopus_train.csv', encoding='utf-8', index=False)
test.to_csv('scopus_test.csv', encoding='utf-8', index=False)

print train.shape
print test.shape

(209169, 7)
(52293, 7)


In [66]:
test.to_json('scopus_test.json', orient='records', lines=True)

In [69]:
train[train.abstract.str.contains('fiscal policy')]

,abstract,id,title,health_sciences,life_sciences,physical_sciences,social_sciences
145194,This article explains (1) the origins of centr...,70CB926DCDDFF00EE07E164668F15415EBECC489,The Origins of Central Banking: Solutions to t...,0,0,0,1
229229,This article explains (1) the origins of centr...,70CB926DCDDFF00EE07E164668F15415EBECC489,The Origins of Central Banking: Solutions to t...,0,0,0,1
61150,This article explains (1) the origins of centr...,70CB926DCDDFF00EE07E164668F15415EBECC489,The Origins of Central Banking: Solutions to t...,0,0,0,1
134771,Summary: The author points out that the review...,40B3AF2933A139BE58EA9A229E03E59FDEA8D302,Modellen en vooroordelen: Het artikel van Prof...,0,0,0,1
169593,"During the five years of inflation, price stab...",3597A9819366963AC8AB8B69FF98B596BDD80BE2,The Supply of Money and Reichsbank Financing o...,0,0,0,1
142670,Abstract: This paper reevaluates the efficacy ...,BB26DD68A885D13A3E60E0EA2FBD953CEC846793,The role of fiscal policy in the St. Louis mod...,0,0,1,1
64265,Colin Hay's combative response is a welcome en...,0C396D5A00F3F2C9BB61A5496E8DF999441E0BE8,"Complexity, Constraint and New Labour's Putati...",0,0,0,1
148307,Colin Hay's combative response is a welcome en...,0C396D5A00F3F2C9BB61A5496E8DF999441E0BE8,"Complexity, Constraint and New Labour's Putati...",0,0,0,1
154274,The recent historiographical revolution in our...,703F5FDC82366291E6D6FEA4DF66F6B1A8E59B8C,From “Fiscal-Military” State to Laissez-faire ...,0,0,0,1
220206,Summary: In this article the debate on the con...,6871D8F6C26275285C92DC6D18240232A2640351,The (UN)importance of public debt: A review essay,0,0,0,1
